In [48]:
import numpy as np
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import cv2
from PIL import Image
%matplotlib inline

In [49]:
def create_model():
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(105, 105, 3))
    x = vgg_conv.output
    #x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(6, activation='softmax')(x)
    
    for layer in vgg_conv.layers:
        layer.trainable=False
    
    model = Model(inputs=vgg_conv.input, outputs=x)
    
    return model

In [50]:
model = create_model()

In [51]:
#rmsprop = RMSprop(lr=0.0001)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    channel_shift_range=4,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [53]:
train_batchsize = 24
test_batchsize = 6

In [54]:
train_generator = train_datagen.flow_from_directory(
        './training_data',
        target_size=(105, 105),
        batch_size=train_batchsize,
        color_mode='rgb',
        class_mode='categorical')

Found 240 images belonging to 6 classes.


In [55]:
validation_generator = validation_datagen.flow_from_directory(
        './testing_data',
        target_size=(105, 105),
        batch_size=test_batchsize,
        class_mode='categorical',
        color_mode='rgb')

Found 60 images belonging to 6 classes.


In [56]:
checkpointer = ModelCheckpoint('logo_model_8.hdf5',save_best_only=True)
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples*5/train_generator.batch_size ,
      epochs=40,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,callbacks=[checkpointer])

Epoch 1/40
50/50 [==============================] - 26s 520ms/step - loss: 1.4293 - acc: 0.7542 - val_loss: 0.4753 - val_acc: 0.9167
Epoch 2/40
50/50 [==============================] - 25s 507ms/step - loss: 0.2375 - acc: 0.9258 - val_loss: 0.4884 - val_acc: 0.8833
Epoch 3/40
50/50 [==============================] - 25s 507ms/step - loss: 0.1575 - acc: 0.9575 - val_loss: 0.5290 - val_acc: 0.9000
Epoch 4/40
50/50 [==============================] - 25s 508ms/step - loss: 0.1228 - acc: 0.9575 - val_loss: 0.5420 - val_acc: 0.8667
Epoch 5/40
50/50 [==============================] - 25s 507ms/step - loss: 0.1088 - acc: 0.9667 - val_loss: 0.6074 - val_acc: 0.9000
Epoch 6/40
50/50 [==============================] - 25s 506ms/step - loss: 0.1136 - acc: 0.9750 - val_loss: 0.4570 - val_acc: 0.8833
Epoch 7/40
50/50 [==============================] - 25s 507ms/step - loss: 0.0574 - acc: 0.9858 - val_loss: 0.6440 - val_acc: 0.9000
Epoch 8/40
50/50 [==============================] - 25s 506ms/step - 